In [ ]:
%load_ext autoreload
%autoreload 2

# Graph aggregate statistics and select similar and dissimilar pairs
We will develop a similarity model for similar pairs; someone will review dissimilar pairs to find nicknames that we will incorporate later

In [ ]:
import matplotlib.pyplot as plt
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
in_path = "s3://familysearch-names/interim/tree-hr-surname-aggr/"
similar_out_path = "s3://nama-data/data/interim/tree-hr-surname-similar.csv"
dissimilar_out_path = "s3://nama-data/data/interim/tree-hr-surname-dissimilar.csv"
# min_freq is 20 for surnames, 5 for given
min_freq = 20

In [ ]:
ProgressBar().register()

In [ ]:
df = dd.read_parquet(in_path).compute()
print(df.shape)

In [ ]:
df.sample(n=10)

In [ ]:
# review low-frequency names
df[(df["total_name_frequency"] < 100) | (df["total_alt_name_frequency"] < 100)].sample(n=25)

In [ ]:
# remove low-frequency names
# freq = df[(df["total_name_frequency"] >= 100) & (df["total_alt_name_frequency"] >= 100)]

In [ ]:

fig, axs = plt.subplots(3, 1, figsize=(16, 16))
axs[0].hist(df["ordered_prob"], bins=20, log=True)
axs[0].set_title("ordered_prob")
axs[1].hist(df["unordered_prob"], bins=20, log=True)
axs[1].set_title("unordered_prob")
axs[2].hist(df["similarity"], bins=20)
axs[2].set_title("similarity")

In [ ]:
plt.figure(figsize=(16, 8))
plt.hist(df["total_name_frequency"], bins=100, range=(1, 1000000), log=True)
plt.title("total_name_frequency")

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(x=df["similarity"], y=df["unordered_prob"])
plt.title("similarity vs probability")
plt.xlabel("similarity")
plt.ylabel("unordered_prob")

In [ ]:
print(df.shape)

In [ ]:
# remove low similarity and low frequency
freq = df[(df["similarity"] > 0.4) & (df["frequency"] > 1)]
print(freq.shape)
freq.sample(n=25)

In [ ]:
# consider removing low-similarity, low-frequency, and low-probability pairs
low_sim_to_remove_indexes = (freq["similarity"] < 0.55) & ((freq["frequency"] < min_freq) | (freq["ordered_prob"] < .10))
low_sim_to_keep_indexes = (freq["similarity"] < 0.55) & ~((freq["frequency"] < min_freq) | (freq["ordered_prob"] < .10))
low_sim_to_remove = freq[low_sim_to_remove_indexes]
low_sim_to_keep = freq[low_sim_to_keep_indexes]

In [ ]:
print(low_sim_to_remove.shape)
low_sim_to_remove.sample(n=25)

In [ ]:
print(low_sim_to_keep.shape)
low_sim_to_keep.sample(n=50)

In [ ]:
# remove noisy pairs as defined above
sim = freq[~low_sim_to_remove_indexes]
print(sim.shape)
# review suspicious pairs
print(sim[sim["similarity"] < 0.55].shape)
sim[sim["similarity"] < 0.55].sample(n=25)

In [ ]:
# review dissimilar but frequent pairs
dis = df[
    (df["similarity"] <= 0.4) & ((df["frequency"] + df["reverse_frequency"]) >= 1000)
]
print(dis.shape)
dis.sample(n=25)

In [ ]:
# write similar and dissimilar pairs
sim.to_csv(similar_out_path, index=False)
dis.to_csv(dissimilar_out_path, index=False)